# ICIJ Offshore Leaks Database &mdash; Address nodes

In the following we want to standardize the management of information related to countries stored in `Address` nodes.

There are $374 955$ `Address` nodes in the dataset.

The following Cypher query outputs the number of different leaks:
```
MATCH (a:Address)
RETURN distinct a.sourceID
```
We see that there are $10$ different leaks: `["Bahamas Leaks", "Offshore Leaks", "Panama Papers", "Paradise Papers - Appleby", "Paradise Papers - Aruba corporate registry", "Paradise Papers - Bahamas corporate registry", "Paradise Papers - Barbados corporate registry", "Paradise Papers - Malta corporate registry", "Paradise Papers - Nevis corporate registry", "Paradise Papers - Samoa corporate registry"]`

In [1]:
from dtgraph import Neo4jGraph, Rule, Transformation
hostname = "localhost"
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="neo4j", username="", password="")

## Experiment 1: Single rule refactoring of Address nodes
We start with `Address` nodes with a `sourceID` of `Paradise Papers - Malta corporate registry`.

These nodes store country information in both `country_code` and `country` attributes.
### Transformation rule
We look at the execution time of the following transformation rule:

In [2]:
panama_malta_address_to_global = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')

In [3]:
#panama_address_to_global._compile(with_diagnose=False)
#print(panama_address_to_global._compiled)

In [4]:
my_transform = Transformation([panama_malta_address_to_global], with_diagnose = False) # no need for diagnose's specific functionalities for the VLDB exp.
rep, tt = 2, 0
for i in range(rep):
    print(f"Iteration {i=}:")
    tt += my_transform.apply_on(graph)
    my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
avg_time = tt/rep
print(f"Average execution time: {avg_time:.3f} ms.")

Iteration i=0:
Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3027 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 618 ms.
Iteration i=1:
Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3242 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 762 ms.
Average execution time: 3134.500 ms.


### Input query
We now look at the execution time and number of rows of the input query.

In [5]:
panama_malta_address_to_global_input_rule = '''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
RETURN a'''
nb_rows, exec_time = graph.query(panama_malta_address_to_global_input_rule)
print(f"Execution time of the input rule: {exec_time} ms.")
print(f"Size of intermediate data: {nb_rows} rows.")

Execution time of the input rule: 858 ms.
Size of intermediate data: 123240 rows.


### Results

Present the statistics and give number of conflicting elements in the output.

In [6]:
_, summary = panama_malta_address_to_global.apply_on(graph)
# clear output data 
my_transform._graph = graph
my_transform.abort(keep_index = True)
num_nodes = summary.counters.nodes_created
num_rels = summary.counters.relationships_created

Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 2792 ms.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 562 ms.


In [7]:
print(f"Ratio between intermediate data and size of the output: {(num_nodes + num_rels) / nb_rows:.3f}")
print(f"Time to compute the intermediary results, per row: {exec_time / nb_rows:.3f} ms.")
print(f"Time to construct the output, per element: {avg_time / (num_nodes + num_rels):.3f} ms.")

Ratio between intermediate data and size of the output: 2.002
Time to compute the intermediary results, per row: 0.007 ms.
Time to construct the output, per element: 0.013 ms.


We now look at the percentage of conflicting elements in the output.

In [8]:
panama_malta_address_to_global_with_diagnose = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE 
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')
my_transform = Transformation([panama_malta_address_to_global_with_diagnose], with_diagnose = True) # We want to retrieve conflicting elements
my_transform.apply_on(graph)
nc, ec = my_transform.diagnose()
my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
print(f"Percentages of conflicting nodes and edges, respectively: {(nc * 100) / num_nodes:.1f}% and {(ec * 100) / num_rels:.1f}%.")

Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3305 ms.
NodeConflicts: There are currently 0 nodes in the database which have a conflict.
EdgeConflicts: There are currently 0 edges in the database which have a conflict.
Abort: Deleted 123433 nodes, deleted 123240 relationships, completed after 739 ms.
Percentages of conflicting nodes and edges, respectively: 0.0% and 0.0%.


Indeed, the following openCypher query's output shows that the FD `country_code -> country` holds on `Address` nodes having the attribute `country_code`.

```
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
WITH a.country_code AS country_code, COUNT(DISTINCT a.country) AS corresponding_country_names
WHERE corresponding_country_names > 1
RETURN country_code, corresponding_country_names
```

## Experiment 2: Whole refactoring of Address nodes
We now consider all the cases in a single transformation.

Address nodes with a `sourceID` different from `Paradise Papers - Malta corporate registry` store country information in both `country_codes` and `countries` attributes.

Actually, the leak `Paradise Papers - Malta corporate registry` were an exception using `country_code` and `country`.

### Transformation rule
We look at the execution time of the transformation consisting of the following rules:

In [9]:
panama_malta_address_to_global = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')
# Create the missing nodes
panama_malta_address_to_global_undefined_country = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})
''')
# For all other leaks, the attributes are in plural form
other_address_to_global = Rule('''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_codes):T_Country {
    name = a.countries
})
''')
other_address_to_global_undefined_countries = Rule('''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})
''')

In [10]:
#other_address_to_global._compile(with_diagnose=False)
#print(other_address_to_global._compiled)

In [11]:
my_transform = Transformation([
    panama_malta_address_to_global, 
    panama_malta_address_to_global_undefined_country,
    other_address_to_global,
    other_address_to_global_undefined_countries,
], with_diagnose = False) # no need for diagnose's specific functionalities for the VLDB exp.
rep, tt = 2, 0
for i in range(rep):
    print(f"Iteration {i=}:")
    tt += my_transform.apply_on(graph)
    my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
avg_time = tt/rep
print(f"Average execution time: {avg_time:.3f} ms.")

Iteration i=0:
Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3130 ms.
Rule: Added 52 labels, created 26 nodes, set 52 properties, created 0 relationships, completed after 123 ms.
Rule: Added 500272 labels, created 250136 nodes, set 1000451 properties, created 250105 relationships, completed after 7802 ms.
Rule: Added 3168 labels, created 1584 nodes, set 3168 properties, created 0 relationships, completed after 191 ms.
Abort: Deleted 375179 nodes, deleted 373345 relationships, completed after 1569 ms.
Iteration i=1:
Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 2826 ms.
Rule: Added 52 labels, created 26 nodes, set 52 properties, created 0 relationships, completed after 103 ms.
Rule: Added 500272 labels, created 250136 nodes, set 1000451 properties, created 25

### Input queries
We now look at the total execution time and total number of rows of the input queries.

In [12]:
nb_rows, exec_time = 0, 0
panama_malta_address_to_global_input_rule = '''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
RETURN a'''
r, e = graph.query(panama_malta_address_to_global_input_rule)
nb_rows += r
exec_time += e
panama_malta_address_to_global_undefined_country_input_rule = '''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NULL
RETURN a'''
r, e = graph.query(panama_malta_address_to_global_undefined_country_input_rule)
nb_rows += r
exec_time += e
other_address_to_global_input_rule = '''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NOT NULL
RETURN a'''
r, e = graph.query(other_address_to_global_input_rule)
nb_rows += r
exec_time += e
other_address_to_global_undefined_countries_input_rule = '''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NULL
RETURN a'''
r, e = graph.query(other_address_to_global_undefined_countries_input_rule)
nb_rows += r
exec_time += e
print(f"Total execution time of the input rules: {exec_time} ms.")
print(f"Total size of intermediate data: {nb_rows} rows.")

Total execution time of the input rules: 2757 ms.
Total size of intermediate data: 374955 rows.


### Results

Present the statistics and give number of conflicting elements in the output.

In [13]:
num_nodes, num_rels = 0, 0
_, summary = panama_malta_address_to_global.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = panama_malta_address_to_global_undefined_country.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = other_address_to_global.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = other_address_to_global_undefined_countries.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
# clear output data 
my_transform._graph = graph
my_transform.abort(keep_index = True)

Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 2956 ms.
Rule: Added 52 labels, created 26 nodes, set 52 properties, created 0 relationships, completed after 106 ms.
Rule: Added 500272 labels, created 250136 nodes, set 1000451 properties, created 250105 relationships, completed after 7969 ms.
Rule: Added 3168 labels, created 1584 nodes, set 3168 properties, created 0 relationships, completed after 192 ms.
Abort: Deleted 375179 nodes, deleted 373345 relationships, completed after 2069 ms.


In [14]:
print(f"Ratio between intermediate data and size of the output: {(num_nodes + num_rels) / nb_rows:.3f}")
print(f"Time to compute the intermediary results, per row: {exec_time / nb_rows:.3f} ms.")
print(f"Time to construct the output, per element: {avg_time / (num_nodes + num_rels):.3f} ms.")

Ratio between intermediate data and size of the output: 1.996
Time to compute the intermediary results, per row: 0.007 ms.
Time to construct the output, per element: 0.015 ms.


In [15]:
panama_malta_address_to_global_with_diagnose = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NOT NULL
GENERATE 
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_code):T_Country {
    name = a.country
})
''')
# Create the missing nodes
panama_malta_address_to_global_undefined_country_with_diagnose = Rule('''
MATCH (a:Address { sourceID: "Paradise Papers - Malta corporate registry" }) 
WHERE a.country_code IS NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})
''')
# For all other leaks, the attributes are in plural form
other_address_to_global_with_diagnose = Rule('''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NOT NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})-[():T_LOCATED]->(y = (a.country_codes):T_Country {
    name = a.countries
})
''')
other_address_to_global_undefined_countries_with_diagnose = Rule('''
MATCH (a:Address)
WHERE a.sourceID <> "Paradise Papers - Malta corporate registry"
  AND a.country_codes IS NULL
GENERATE
(x = (a):T_Address {
    source = a.sourceID
})
''')
my_transform = Transformation([
    panama_malta_address_to_global_with_diagnose, 
    panama_malta_address_to_global_undefined_country_with_diagnose,
    other_address_to_global_with_diagnose,
    other_address_to_global_undefined_countries_with_diagnose,
], with_diagnose = True) # no need for diagnose's specific functionalities for the VLDB exp.
my_transform.apply_on(graph)
nc, ec = my_transform.diagnose()
my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
print(f"Percentages of conflicting nodes and edges, respectively: {(nc * 100) / num_nodes:.3f}% and {(ec * 100) / num_rels:.3f}%.")

Index: Added 0 index, completed after 1 ms.
Rule: Added 246866 labels, created 123433 nodes, set 493153 properties, created 123240 relationships, completed after 3411 ms.
Rule: Added 52 labels, created 26 nodes, set 52 properties, created 0 relationships, completed after 122 ms.
Rule: Added 500278 labels, created 250136 nodes, set 1000451 properties, created 250105 relationships, completed after 8017 ms.
Rule: Added 3168 labels, created 1584 nodes, set 3168 properties, created 0 relationships, completed after 192 ms.
NodeConflicts: There are currently 6 nodes in the database which have a conflict.
  (:T_Country {}) has a conflict on attributes ['name'].
  (:T_Country {}) has a conflict on attributes ['name'].
  (:T_Country {}) has a conflict on attributes ['name'].
  (:T_Country {}) has a conflict on attributes ['name'].
  (:T_Country {}) has a conflict on attributes ['name'].
  (:T_Country {}) has a conflict on attributes ['name'].
EdgeConflicts: There are currently 0 edges in the dat

Let us investigate using the following openCypher query what are the `Address` nodes having the attribute `country_codes` on which the FD `country_codes -> countries` does not hold.

```
MATCH (a:Address) 
WHERE a.country_codes IS NOT NULL
WITH a.country_codes AS country_codes, collect(DISTINCT a.countries) AS corresponding_countries_names
WHERE size(corresponding_countries_names) > 1
RETURN country_codes, corresponding_countries_names
```
which returns:

```
╒═════════════╤═════════════════════════════════════════════════════╕
│country_codes│corresponding_countries_names                        │
╞═════════════╪═════════════════════════════════════════════════════╡
│"KOR"        │["South Korea", "Korea, Republic of"]                │
├─────────────┼─────────────────────────────────────────────────────┤
│"TWN"        │["Taiwan", "Taiwan, Province of China"]              │
├─────────────┼─────────────────────────────────────────────────────┤
│"VEN"        │["Venezuela", "Venezuela, Bolivarian Republic of"]   │
├─────────────┼─────────────────────────────────────────────────────┤
│"VGB"        │["British Virgin Islands", "Virgin Islands, British"]│
├─────────────┼─────────────────────────────────────────────────────┤
│"VIR"        │["U.S. Virgin Islands", "Virgin Islands, U.S."]      │
└─────────────┴─────────────────────────────────────────────────────┘
```
We see that these entries refer to the same country using similar names.  This might suggest that the FD is meaningful, but data cleaning is probably necessary.

## Experiment 3: Uniformizing intermediaries' address information
In this dataset, it may be difficult to find information about the addresses of the $25 745$ intermediaries.  
Indeed, their addresses can be stored in three different ways:
1. `(i:Intermediary)-[:registered_address]->(a:Address)` which concern $10 496$ such intermediaries. In that case, `i.address` is never defined.
2. `(i:Intermediary)` where `i.address` is defined and not empty. This concerns $8 643$ such intermediaries.
3. `(i:Intermediary)` where `i.address` is defined and empty. This concerns $3 978$ such intermediaries.  
  In this case, the following query retrieves the $3 970$ intermediaries where `i.address` is defined and empty and connected to an entity having its address set.  
  This is consistent with [the specification of the data](https://offshoreleaks-data.icij.org/offshoreleaks/neo4j/guide/datashape.html) which states that:  
  *Each Entity node has the address property only when the address is the same as the one of the intermediary. Otherwise, the registered address information is stored in the address node connected to this Entity node through a REGISTERED_ADDRESS relationship.*
  ```
  MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
  WHERE i.address IS NOT NULL AND i.address = "" 
  WITH i, collect(DISTINCT e.address) AS e_addresses 
  WHERE size(e_addresses) > 0
  RETURN i, e_addresses
  ```  

Hence using rules we can uniformize the presentation of address information for the $10 496 + 8643 + 3 970 = 23 109$  intermediaries for which an address is known (out of $25 745$ intermediaries).

### Transformation rule
We look at the execution time of the transformation consisting of the following rules:

In [43]:
intermediary_registered_address = Rule('''
MATCH (i:Intermediary)-[:registered_address]->(a:Address)
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (a):T_Address)
''')
intermediary_not_empty_address = Rule('''
MATCH (i:Intermediary)
WHERE i.address IS NOT NULL AND i.address <> "" AND i.country_codes IS NOT NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (i.address):T_Address {
  source = i.sourceID
}),
(y)-[():T_LOCATED]->(z = (i.country_codes):T_Country {
    name = i.countries
})
''')
intermediary_not_empty_address_undefined_countries = Rule('''
MATCH (i:Intermediary)
WHERE i.address IS NOT NULL AND i.address <> "" AND i.country_codes IS NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (i.address):T_Address {
  source = i.sourceID
})
''')
intermediary_empty_address = Rule('''
MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
WHERE i.address IS NOT NULL AND i.address = "" AND e.country_codes IS NOT NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (e.address):T_Address {
  source = e.sourceID
}),
(y)-[():T_LOCATED]->(z = (e.country_codes):T_Country {
    name = e.countries
})
''')
intermediary_empty_address_undefined_countries = Rule('''
MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
WHERE i.address IS NOT NULL AND i.address = "" AND e.country_codes IS NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (e.address):T_Address {
  source = e.sourceID
})
''')

In [44]:
#intermediary_not_empty_address._compile()
#print(intermediary_not_empty_address._compiled)

In [45]:
my_transform = Transformation([
    intermediary_registered_address, 
    intermediary_not_empty_address,
    intermediary_not_empty_address_undefined_countries,
    intermediary_empty_address,
    intermediary_empty_address_undefined_countries
], with_diagnose = False) # no need for diagnose's specific functionalities for the VLDB exp.
rep, tt = 2, 0
for i in range(rep):
    print(f"Iteration {i=}:")
    tt += my_transform.apply_on(graph)
    my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
avg_time = tt/rep
print(f"Average execution time: {avg_time:.3f} ms.")

Iteration i=0:
Index: Added 0 index, completed after 1 ms.
Rule: Added 40386 labels, created 20193 nodes, set 30689 properties, created 10496 relationships, completed after 845 ms.
Rule: Added 34950 labels, created 17475 nodes, set 51952 properties, created 17237 relationships, completed after 575 ms.
Rule: Added 92 labels, created 46 nodes, set 92 properties, created 23 relationships, completed after 673 ms.
Rule: Added 16650 labels, created 8325 nodes, set 103031 properties, created 8740 relationships, completed after 2868 ms.
Rule: Added 106 labels, created 53 nodes, set 201 properties, created 28 relationships, completed after 1479 ms.
Abort: Deleted 46092 nodes, deleted 36524 relationships, completed after 415 ms.
Iteration i=1:
Index: Added 0 index, completed after 0 ms.
Rule: Added 40386 labels, created 20193 nodes, set 30689 properties, created 10496 relationships, completed after 608 ms.
Rule: Added 34950 labels, created 17475 nodes, set 51952 properties, created 17237 relatio

### Input queries
We now look at the total execution time and total number of rows of the input queries.

In [46]:
nb_rows, exec_time = 0, 0
intermediary_registered_address_input_rule = '''
MATCH (i:Intermediary)-[:registered_address]->(a:Address)
RETURN i, a'''
r, e = graph.query(intermediary_registered_address_input_rule)
nb_rows += r
exec_time += e
intermediary_not_empty_address_input_rule = '''
MATCH (i:Intermediary)
WHERE i.address IS NOT NULL AND i.address <> "" AND i.country_codes IS NOT NULL
RETURN i'''
r, e = graph.query(intermediary_not_empty_address_input_rule)
nb_rows += r
exec_time += e
intermediary_not_empty_address_undefined_countries_input_rule = '''
MATCH (i:Intermediary)
WHERE i.address IS NOT NULL AND i.address <> "" AND i.country_codes IS NULL
RETURN i'''
r, e = graph.query(intermediary_not_empty_address_undefined_countries_input_rule)
nb_rows += r
exec_time += e
intermediary_empty_address_input_rule = '''
MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
WHERE i.address IS NOT NULL AND i.address = "" AND e.country_codes IS NOT NULL
RETURN i, e'''
r, e = graph.query(intermediary_empty_address_input_rule)
nb_rows += r
exec_time += e
intermediary_empty_address_undefined_countries_input_rule = '''
MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
WHERE i.address IS NOT NULL AND i.address = "" AND e.country_codes IS NULL
RETURN i, e'''
r, e = graph.query(intermediary_empty_address_undefined_countries_input_rule)
nb_rows += r
exec_time += e
print(f"Total execution time of the input rules: {exec_time} ms.")
print(f"Total size of intermediate data: {nb_rows} rows.")

Total execution time of the input rules: 3553 ms.
Total size of intermediate data: 62242 rows.


### Results

Present the statistics and give number of conflicting elements in the output.

In [47]:
num_nodes, num_rels = 0, 0
_, summary = intermediary_registered_address.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = intermediary_not_empty_address.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = intermediary_not_empty_address_undefined_countries.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = intermediary_empty_address.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
_, summary = intermediary_empty_address_undefined_countries.apply_on(graph)
num_nodes += summary.counters.nodes_created
num_rels += summary.counters.relationships_created
# clear output data 
my_transform._graph = graph
my_transform.abort(keep_index = True)

Rule: Added 40386 labels, created 20193 nodes, set 30689 properties, created 10496 relationships, completed after 745 ms.
Rule: Added 34950 labels, created 17475 nodes, set 51952 properties, created 17237 relationships, completed after 481 ms.
Rule: Added 92 labels, created 46 nodes, set 92 properties, created 23 relationships, completed after 567 ms.
Rule: Added 16650 labels, created 8325 nodes, set 103031 properties, created 8740 relationships, completed after 2631 ms.
Rule: Added 106 labels, created 53 nodes, set 201 properties, created 28 relationships, completed after 1491 ms.
Abort: Deleted 46092 nodes, deleted 36524 relationships, completed after 391 ms.


In [48]:
print(f"Ratio between intermediate data and size of the output: {(num_nodes + num_rels) / nb_rows:.3f}")
print(f"Time to compute the intermediary results, per row: {exec_time / nb_rows:.3f} ms.")
print(f"Time to construct the output, per element: {avg_time / (num_nodes + num_rels):.3f} ms.")

Ratio between intermediate data and size of the output: 1.327
Time to compute the intermediary results, per row: 0.057 ms.
Time to construct the output, per element: 0.072 ms.


In [49]:
intermediary_registered_address_with_diagnose = Rule('''
MATCH (i:Intermediary)-[:registered_address]->(a:Address)
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (a):T_Address)
''')
intermediary_not_empty_address_with_diagnose = Rule('''
MATCH (i:Intermediary)
WHERE i.address IS NOT NULL AND i.address <> "" AND i.country_codes IS NOT NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (i.address):T_Address {
  source = i.sourceID
}),
(y)-[():T_LOCATED]->(z = (i.country_codes):T_Country {
    name = i.countries
})
''')
intermediary_not_empty_address_undefined_countries_with_diagnose = Rule('''
MATCH (i:Intermediary)
WHERE i.address IS NOT NULL AND i.address <> "" AND i.country_codes IS NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (i.address):T_Address {
  source = i.sourceID
})
''')
intermediary_empty_address_with_diagnose = Rule('''
MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
WHERE i.address IS NOT NULL AND i.address = "" AND e.country_codes IS NOT NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (e.address):T_Address {
  source = e.sourceID
}),
(y)-[():T_LOCATED]->(z = (e.country_codes):T_Country {
    name = e.countries
})
''')
intermediary_empty_address_undefined_countries_with_diagnose = Rule('''
MATCH (i:Intermediary)-[:intermediary_of]->(e:Entity) 
WHERE i.address IS NOT NULL AND i.address = "" AND e.country_codes IS NULL
GENERATE
(x = (i):T_Intermediary)-[():T_REGISTERED_ADDRESS]->(y = (e.address):T_Address {
  source = e.sourceID
})
''')
my_transform = Transformation([
    intermediary_registered_address_with_diagnose, 
    intermediary_not_empty_address_with_diagnose,
    intermediary_not_empty_address_undefined_countries_with_diagnose,
    intermediary_empty_address_with_diagnose,
    intermediary_empty_address_undefined_countries_with_diagnose,
], with_diagnose = True) # no need for diagnose's specific functionalities for the VLDB exp.
my_transform.apply_on(graph)
nc, ec = my_transform.diagnose()
my_transform.abort(keep_index = True) # Strange bug: indexes created with the Neo4j Python driver cannot be used in subsequent query plans
print(f"Percentages of conflicting nodes and edges, respectively: {(nc * 100) / num_nodes:.3f}% and {(ec * 100) / num_rels:.3f}%.")

Index: Added 0 index, completed after 0 ms.
Rule: Added 40386 labels, created 20193 nodes, set 30689 properties, created 10496 relationships, completed after 633 ms.
Rule: Added 34950 labels, created 17475 nodes, set 51952 properties, created 17237 relationships, completed after 709 ms.
Rule: Added 92 labels, created 46 nodes, set 92 properties, created 23 relationships, completed after 675 ms.
Rule: Added 16650 labels, created 8325 nodes, set 103031 properties, created 8740 relationships, completed after 2805 ms.
Rule: Added 106 labels, created 53 nodes, set 201 properties, created 28 relationships, completed after 1448 ms.
NodeConflicts: There are currently 0 nodes in the database which have a conflict.
EdgeConflicts: There are currently 0 edges in the database which have a conflict.
Abort: Deleted 46092 nodes, deleted 36524 relationships, completed after 435 ms.
Percentages of conflicting nodes and edges, respectively: 0.000% and 0.000%.
